In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
from pathlib import Path
import sys
from time import time
import numpy as np
import pandas as pd

import sklearn
from collections import OrderedDict
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

SEED = 42

# Load data

In [2]:
datadir = Path('../../data/yitan/Data')
ccl_folds_dir = Path('../../data/yitan/CCL_10Fold_Partition')
pdm_folds_dir = Path('../../data/yitan/PDM_10Fold_Partition')
fea_data_name = 'CCL_PDM_TransferLearningData_rmFactor_0.0_ddNorm_std.pkl'

In [3]:
# Un-pickle files
import _pickle as cp

pkl_file = open(datadir/fea_data_name, 'rb')
res = cp.load(pkl_file)
ccl = cp.load(pkl_file)
drg = cp.load(pkl_file)
pkl_file.close()

In [4]:
print('res ', res.shape)
print('ccl ', ccl.shape)
print('drg ', drg.shape)

res  (708662, 5)
ccl  (1430, 4582)
drg  (1402, 4392)


# First look at the data

In [5]:
display(res[:2])
display(ccl.iloc[:2, :7])
display(drg.iloc[:2, :7])

,SOURCE,ccl_name,ctrpDrugID,area_under_curve,groupID
0,CCLE,CCL_61,Drug_11,0.7153,0.0000
1,CCLE,CCL_61,Drug_1,0.9579,0.9164


,geneGE_AARS,geneGE_ABCB6,geneGE_ABCC5,geneGE_ABCF1,geneGE_ABCF3,geneGE_ABHD4,geneGE_ABHD6
CCL_1,-0.125161,-0.400237,-0.960208,0.575207,-0.468406,-0.136257,0.083319
CCL_10,-0.217106,0.354776,-1.164841,0.328071,-0.735267,0.232990,-0.174979


,DD_MW|num,DD_AMW|num,DD_Sv|num,DD_Se|num,DD_Sp|num,DD_Si|num,DD_Mv|num
Drug_1,0.123446,0.526234,-0.072180,-0.088861,-0.058460,-0.083100,-0.009539
Drug_10,0.053188,1.966100,-0.333843,-0.379081,-0.359584,-0.398841,1.172374


In [6]:
res.groupby('SOURCE').agg({'ccl_name': 'nunique', 'ctrpDrugID': 'nunique'}).reset_index()

,SOURCE,ccl_name,ctrpDrugID
0,CCLE,474,24
1,CTRP,812,494
2,GDSC,670,238
3,NCI60,59,987
4,PDM,473,18
5,gCSI,357,16


In [7]:
# Check normalization
print(ccl.mean(axis=0).mean())
print(ccl.var(axis=0).mean())

print(drg.mean(axis=0).mean())
print(drg.var(axis=0).mean())

-5.949566358842101e-18
1.000699790062981
0.1652929135842889
0.5961096596718728


# Update dfs

In [8]:
# Bring in the row index
res = res.reset_index()
res = res.rename(columns={'index': 'idx', 'SOURCE': 'src', 'area_under_curve': 'auc'})
res[:2]

,idx,src,ccl_name,ctrpDrugID,auc,groupID
0,0,CCLE,CCL_61,Drug_11,0.7153,0.0000
1,1,CCLE,CCL_61,Drug_1,0.9579,0.9164


In [9]:
# Scale
def scale_df(df):
    index = df.index
    columns = df.columns
    scaler = StandardScaler()
    df = pd.DataFrame( scaler.fit_transform(df), index=index, columns=columns ).astype(np.float32)
    return df

# ccl = scale_df(ccl)
# drg = scale_df(drg)

# Where are the genomic features coming from?
Gene expression is coming from CCLE or NCI60.

In [10]:
# ccl_ = cell.reset_index().rename(columns={'index': 'ccl_name'})

# display(ccl_.iloc[:2, :5])
# display(res[:2])

In [11]:
# aa = pd.merge(ccl_[['ccl_name']], res[['SOURCE', 'ccl_name']], on='ccl_name', how='inner')
# # aa = aa.drop_duplicates().reset_index(drop=True)
# print(aa.shape)
# print(aa['SOURCE'].value_counts())

# bb = aa[aa['ccl_name']=='CCL_1']['SOURCE'].value_counts()
# print(bb)

In [12]:
# del ccl_, aa, bb

# What features are available?

In [13]:
# def cnt_fea(df, fea_sep='_', verbose=True):
#     """ Count the number of features per feature type. """
#     dct = {}
#     unq_prfx = df.columns.map(lambda x: x.split(fea_sep)[0]).unique() # unique feature prefixes
#     for prfx in unq_prfx:
#         fea_type_cols = [c for c in df.columns if (c.split(fea_sep)[0]) in prfx] # all fea names of specific type
#         dct[prfx] = len(fea_type_cols)
#     if verbose: print(dct)
#     return dct

# cnt_fea(ccl, fea_sep='_');
# cnt_fea(drg, fea_sep='_');

In [14]:
# def extract_subset_fea(df, fea_list, fea_sep='_'):
#     """ Extract features based feature prefix name. """
#     fea = [c for c in df.columns if (c.split(fea_sep)[0]) in fea_list]
#     df = df[fea]
#     return df

# tmp = extract_subset_fea(ccl, fea_list=['geneGE', 'c2cpMaxGE'], fea_sep='_')
# cnt_fea(tmp, fea_sep='_', verbose=True);

In [15]:
# def extract_unq_fea_dfs(df, fea_sep='_'):
#     """ Generate dict where each element is a separate df with unique feature type. """
#     dct_fea_prfx = cnt_fea(df, fea_sep=fea_sep)
#     dct_dfs = {}
#     for k in dct_fea_prfx.keys():
#         fea_type_cols = [c for c in df.columns if (c.split(fea_sep)[0]) in k]
#         dct_dfs[k] = df[fea_type_cols]
#     return dct_dfs

# # Cell dfs
# ccl_dct = extract_unq_fea_dfs(ccl, fea_sep='_')
# display(ccl_dct['geneGE'].shape)
# display(ccl_dct['c2cpMaxGE'].shape)
# display(ccl_dct['c2cpMinGE'].shape)

# # Drug dfs
# drg_dct = extract_unq_fea_dfs(drg, fea_sep='_')
# display(drg_dct['DD'].shape)
# display(drg_dct['ECFP'].shape)
# display(drg_dct['PFP'].shape)

# Merge features and target into a single df

In [10]:
def cnt_fea(df, fea_sep='_', verbose=True):
    """ Count the number of features per feature type. """
    dct = {}
    unq_prfx = df.columns.map(lambda x: x.split(fea_sep)[0]).unique() # unique feature prefixes
    for prfx in unq_prfx:
        fea_type_cols = [c for c in df.columns if (c.split(fea_sep)[0]) in prfx] # all fea names of specific type
        dct[prfx] = len(fea_type_cols)
    if verbose: print(dct)
    return dct

def extract_subset_fea(df, fea_list, fea_sep='_'):
    """ Extract features based feature prefix name. """
    fea = [c for c in df.columns if (c.split(fea_sep)[0]) in fea_list]
    df = df[fea]
    return df

In [ ]:
# Args
src = 'GDSC'
fold = 0
ccl_fea_list = ['geneGE']
drg_fea_list = ['DD']
fea_sep = '_'
# fea_float_dtype = fea_float_dtype

In [11]:
res_df_ = res.copy()
ccl_df_ = ccl.copy()
drg_df_ = drg.copy()

# Get lists of ccl names based on src and fold
ids_path = ccl_folds_dir/f'{src}/cv_{fold}' # 'TestList.txt'
tr_ids_list = pd.read_csv(ids_path/'TrainList.txt', header=None).squeeze().values
vl_ids_list = pd.read_csv(ids_path/'ValList.txt', header=None).squeeze().values
te_ids_list = pd.read_csv(ids_path/'TestList.txt', header=None).squeeze().values

# Show how much is left for train, val, and test
tr_sz, vl_sz, te_sz = len(tr_ids_list), len(vl_ids_list), len(te_ids_list)
sz = tr_sz + vl_sz + te_sz
print(f'Train portion: {tr_sz/sz:.2f}')

# Retain specific source
print('Full dataset:', res_df_.shape)
res_df_ = res_df_[ res_df_['src'].isin([src]) ]
print(f'Only {src}: ', res_df_.shape)

# Create res dfs for train and val
res_tr = res_df_[ res_df_['ccl_name'].isin( tr_ids_list ) ]
res_vl = res_df_[ res_df_['ccl_name'].isin( vl_ids_list ) ]
print('\nres_tr.shape:', res_tr.shape)
print('res_vl.shape:', res_vl.shape)

# Extract specific types of features
print('\nExtract fea types ...')
cnt_fea(ccl_df_);
ccl_df_ = extract_subset_fea(df=ccl_df_, fea_list=ccl_fea_list, fea_sep=fea_sep)
cnt_fea(ccl_df_);

cnt_fea(drg_df_);
drg_df_ = extract_subset_fea(df=drg_df_, fea_list=drg_fea_list, fea_sep=fea_sep)
cnt_fea(drg_df_);

Train portion: 0.80
Full dataset: (708662, 6)
Only GDSC:  (135568, 6)

res_tr.shape: (108481, 6)
res_vl.shape: (13507, 6)

Extract fea types ...
{'geneGE': 1927, 'c2cpMaxGE': 1328, 'c2cpMinGE': 1327}
{'geneGE': 1927}
{'DD': 2344, 'ECFP': 1024, 'PFP': 1024}
{'DD': 2344}


In [12]:
# Bring the labels in, in order to merge on
ccl_df_ = ccl_df_.reset_index().rename(columns={'index': 'ccl_name'})
drg_df_ = drg_df_.reset_index().rename(columns={'index': 'ctrpDrugID'})

In [13]:
# Merge data
def merge_dfs(res_df, ccl_df, drg_df):
    """ Merge the following dfs: response, ccl fea, drug fea """
    mrg_df = pd.merge(res_df, ccl_df, on='ccl_name', how='inner')
    mrg_df = pd.merge(mrg_df, drg_df, on='ctrpDrugID', how='inner')
    return mrg_df

mrg_tr = merge_dfs(res_tr, ccl_df_, drg_df_)
mrg_vl = merge_dfs(res_vl, ccl_df_, drg_df_)

print('\nMerge ...')
print('mrg_tr.shape:', mrg_tr.shape)
print('mrg_vl.shape:', mrg_vl.shape)

mrg_tr.shape: (108481, 4277)
mrg_vl.shape: (13507, 4277)


In [14]:
# Extract x and y
xtr = extract_subset_fea(mrg_tr, fea_list = ccl_fea_list + drg_fea_list, fea_sep=fea_sep)
ytr = mrg_tr[['auc']]

xvl = extract_subset_fea(mrg_vl, fea_list = ccl_fea_list + drg_fea_list, fea_sep=fea_sep)
yvl = mrg_vl[['auc']]

print('\nExtract x and y ...')
print('xtr.shape:', xtr.shape)
print('xvl.shape:', xvl.shape)
print('ytr.shape:', ytr.shape)
print('yvl.shape:', yvl.shape)

xtr.shape: (108481, 4271)
xvl.shape: (13507, 4271)


In [16]:
# Dump to file
xtr.to_parquet(datadir/f'{src.lower()}_xtr.parquet')
ytr.to_parquet(datadir/f'{src.lower()}_ytr.parquet')

xvl.to_parquet(datadir/f'{src.lower()}_xvl.parquet')
yvl.to_parquet(datadir/f'{src.lower()}_yvl.parquet')

# Define Dataset

In [10]:
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

print(torch.__version__)

1.1.0


In [11]:
print(res.shape)
print(ccl.shape)
print(drg.shape)

(708662, 6)
(1430, 4582)
(1402, 4392)


In [22]:
# ------------------------------
#  Test code of pytroch dataset
# ------------------------------

# res_df = res.copy()
# drg_df = drg.copy()
# ccl_df = ccl.copy()

# tr_ph = 'train'
# fold = 0
# src = 'CCLE'
# ccl_fea_list = ['geneGE']
# drg_fea_list = ['DD']

# res_df = res_df[ res_df['SOURCE'].isin([src]) ]
# drg_df = extract_subset_fea(drg_df, drg_fea_list, fea_sep='_')
# ccl_df = extract_subset_fea(ccl_df, ccl_fea_list, fea_sep='_')

# path = ccl_folds_dir/f'{src}/cv_{fold}'

# res_arr = res_df.values
# drg_fea_dct = {idx: row.values for idx, row in drg_df.iterrows()}
# ccl_fea_dct = {idx: row.values for idx, row in ccl_df.iterrows()}

In [12]:
class Dataset_Raw(Dataset):
    # discuss.pytorch.org/t/data-processing-as-a-batch-way/14154
    # github.com/utkuozbulak/pytorch-custom-dataset-examples#incorporating-pandas
    # nbviewer.jupyter.org/github/FraPochetti/KagglePlaygrounds/blob/master/NYC%20Taxi%20Fares%20Prediction.ipynb
    def __init__(self,
                 res_df: pd.DataFrame,
                 ccl_df: pd.DataFrame,
                 drg_df: pd.DataFrame,
                 ccl_folds_dir: str,
                 src: str,
                 fold: int,
                 tr_ph: str,
                 ccl_fea_list: list=None,
                 drg_fea_list: list=None,
                 fea_sep: str='_',
                 fea_float_dtype: type=torch.float32,
                 # target_float_dtype: type=torch.float64,
                 # drg_dsc_preproc: str=None,  # TODO
                 # ccl_rna_preproc: str=None,  # TODO
                 verbose: bool=True):
        """ 
        Define pytorch dataset for drug response prediction.
        Args:
            res_df (pd.DataFrame) : drug response df
            ccl_df (pd.DataFrame) : cell feature df
            drg_df (pd.DataFrame) : drug feature df
            ccl_folds_dir (str) : folder path that contains cv patitions in text files
            src (str) : source name
            fold (int) : fold index
            tr_ph (str) : training phase ('tr', 'vl', 'te')
            ccl_fea_list (list) : list of prefixes of cell features to retain
            drg_fea_list (list) : list of prefixes of drug features to retain
            fea_sep (str) : separator of feature prefix that indicates type and feature name
            fea_float_dtype (type) : float precision for features
            drg_dsc_preproc (str) : TODO: not implemented
            cell_rna_preproc (str) : TODO: not implemented
            verbose : bool=True
        """
        # ============================================
        # Initialize
        # ============================================
        self.ccl_folds_dir = ccl_folds_dir
        self.src = src
        self.fold = fold
        self.tr_ph = tr_ph.lower()
        self.ccl_fea_list = ccl_fea_list
        self.drg_fea_list = drg_fea_list
        self.fea_sep = fea_sep
        self.fea_float_dtype = fea_float_dtype

        # ============================================
        # Get the ccl names
        # ============================================
        if self.tr_ph in ['tr', 'train', 'training']:
            self.ids_fname = 'TrainList.txt'
        elif self.tr_ph in ['vl', 'val', 'validation']:
            self.ids_fname = 'ValList.txt'
        elif self.tr_ph in ['te', 'test', 'testing']:
            self.ids_fname = 'TestList.txt'
        else:
            raise ValueError('Must specify valid `tr_ph` argument.')
            
        self.ids_path = self.ccl_folds_dir/f'{self.src}/cv_{self.fold}'/self.ids_fname # 'TestList.txt'        
        self.ids_list = pd.read_csv(self.ids_path, header=None).squeeze().values
        
        # ============================================
        # Load dfs
        # ============================================
        res_df = res_df[ res_df['src'].isin([src]) ]  # extract responses of specific source
        self.res_df = res_df[ res_df['ccl_name'].isin( self.ids_list ) ]  # extract responses of specific ccl samples
        
        # Extract specific types of features
        self.ccl_df = ccl_df if self.ccl_fea_list is None else self.extract_subset_fea(ccl_df, fea_list=self.ccl_fea_list, fea_sep=self.fea_sep)
        self.drg_df = drg_df if self.drg_fea_list is None else self.extract_subset_fea(drg_df, fea_list=self.drg_fea_list, fea_sep=self.fea_sep)
        
        # ============================================
        # Public attributes
        # ============================================
        self.cells = self.res_df['ccl_name'].unique().tolist()    # unique cells
        self.drugs = self.res_df['ctrpDrugID'].unique().tolist()  # unique drugs
        self.num_records = len(self.res_df)
        self.ccl_dim = self.ccl_df.shape[1]
        self.drg_dim = self.drg_df.shape[1]
        
        self.ccl_fea_cnt = self.cnt_fea(self.ccl_df, fea_sep=self.fea_sep, verbose=False)
        self.drg_fea_cnt = self.cnt_fea(self.drg_df, fea_sep=self.fea_sep, verbose=False)
        
        # ============================================
        # Convert dfs to arrays and dict for faster access
        # ============================================
        # self.res_arr = self.res_df.values
        # self.ccl_fea_dct = {idx: row.values for idx, row in self.ccl_df.iterrows()}
        # self.drg_fea_dct = {idx: row.values for idx, row in self.drg_df.iterrows()}
        # TODO: does the values must be pytorch tensors??
        self.res_arr = self.res_df.values
        self.ccl_fea_dct = {idx: torch.tensor(row.values, dtype=fea_float_dtype) for idx, row in self.ccl_df.iterrows()}
        self.drg_fea_dct = {idx: torch.tensor(row.values, dtype=fea_float_dtype) for idx, row in self.drg_df.iterrows()}

        # ============================================
        # Summary
        # ============================================
        if verbose:
            print('=' * 80)
            print(f'Data source: {self.src}')
            print(f'Phase: {tr_ph}')
            print(f'Data points: {self.num_records}')
            print(f'Unique cells: {len(self.cells)}')
            print(f'Unique drugs: {len(self.drugs)}')
            print(f'ccl_df.shape: {self.ccl_df.shape}')
            print(f'drg_df.shape: {self.drg_df.shape}')
            print(f'Cell features: {self.ccl_fea_cnt}')
            print(f'Drug features: {self.drg_fea_cnt}')
            

    def __len__(self):
        return len(self.res_arr)

    
    def __getitem__(self, index):
        """ 
        Ref: github.com/xduan7/UnoPytorch/blob/master/utils/datasets/drug_resp_dataset.py
        Look for __getitem__ in DrugRespDataset
        
        res indices: [idx, src, ccl_name, ctrpDrugID, auc, groupID]
        """
        res = self.res_arr[index]
        
        idx = res[0]
        src = res[1]
        ccl_id = res[2]
        drg_id = res[3]
        auc = res[4]
        grp_id = res[5]
        
        ccl_fea = self.ccl_fea_dct[ccl_id]
        drg_fea = self.drg_fea_dct[drg_id]
        
        # Cast values
        # ccl_fea = ccl_fea.astype(np.float32)
        # drg_fea = drg_fea.astype(np.float32)
        
        # return ccl_fea, drg_fea, auc
        return idx, src, ccl_id, drg_id, auc, ccl_fea, drg_fea
    
    
    def extract_subset_fea(self, df, fea_list, fea_sep='_'):
        """ Extract features based feature prefix name. """
        fea = [c for c in df.columns if (c.split(fea_sep)[0]) in fea_list]
        df = df[fea]
        return df    
    
    
    def cnt_fea(self, df, fea_sep='_', verbose=True):
        """ Count the number of features per feature type. """
        dct = {}
        unq_prfx = df.columns.map(lambda x: x.split(fea_sep)[0]).unique() # unique feature prefixes
        for prfx in unq_prfx:
            fea_type_cols = [c for c in df.columns if (c.split(fea_sep)[0]) in prfx] # all fea names of specific type
            dct[prfx] = len(fea_type_cols)
        if verbose: print(dct)
        return dct

# Create datasets

In [13]:
ccl_fea_list = ['geneGE']
drg_fea_list = ['DD']

ds_kwargs = {
    'res_df': res,
    'ccl_df': ccl,
    'drg_df': drg,
    'ccl_folds_dir': ccl_folds_dir,
    'src': 'GDSC',
    'ccl_fea_list': ccl_fea_list,
    'drg_fea_list': drg_fea_list,
    'fea_sep': '_'}

fold = 0
tr_ds = Dataset_Raw(tr_ph='tr', fold=fold, **ds_kwargs)
vl_ds = Dataset_Raw(tr_ph='vl', fold=fold, **ds_kwargs)
te_ds = Dataset_Raw(tr_ph='te', fold=fold, **ds_kwargs)

Data source: GDSC
Phase: tr
Data points: 108481
Unique cells: 536
Unique drugs: 238
ccl_df.shape: (1430, 1927)
drg_df.shape: (1402, 2344)
Cell features: {'geneGE': 1927}
Drug features: {'DD': 2344}
Data source: GDSC
Phase: vl
Data points: 13507
Unique cells: 67
Unique drugs: 238
ccl_df.shape: (1430, 1927)
drg_df.shape: (1402, 2344)
Cell features: {'geneGE': 1927}
Drug features: {'DD': 2344}
Data source: GDSC
Phase: te
Data points: 13580
Unique cells: 67
Unique drugs: 238
ccl_df.shape: (1430, 1927)
drg_df.shape: (1402, 2344)
Cell features: {'geneGE': 1927}
Drug features: {'DD': 2344}


# Create data loaders
Data loaders for training/validation in github.com/xduan7/UnoPytorch/blob/master/uno_pytorch.py

In [14]:
batch_size = 32
n_jobs = 4
tr_loader_kwargs = {'batch_size': batch_size, 'shuffle': True,  'num_workers': n_jobs}
vl_loader_kwargs = {'batch_size': 4*batch_size, 'shuffle': False, 'num_workers': n_jobs}
te_loader_kwargs = {'batch_size': 4*batch_size, 'shuffle': False, 'num_workers': n_jobs}

tr_loader = DataLoader(tr_ds, **tr_loader_kwargs)
vl_loader = DataLoader(vl_ds, **vl_loader_kwargs)
te_loader = DataLoader(te_ds, **te_loader_kwargs)

### --- Test --- generate a single batch from dataloader

In [26]:
# ret = next(iter(tr_loader))  # (idx, SOURCE, ccl_id, drg_id, auc, ccl_fea, drg_fea)

# idx = ret[0]
# src = ret[1]
# cell_id = ret[2]
# drug_id = ret[3]
# auc = ret[4]
# ccl_fea = ret[5]
# drg_fea = ret[6]

# print('ccl_fea:', ccl_fea.shape)
# print('drg_fea:', drg_fea.shape)
# display(ret)

In [27]:
# # print( sum(tr_ds.res_df['ccl_name']==ret[2][0]) )
# # print( sum(tr_ds.res_df['ctrpDrugID']==ret[3][0]) )
# # sum( (tr_ds.res_df['ccl_name']==ret[2][0]) & (tr_ds.res_df['ctrpDrugID']==ret[3][0]) )

# tr_ds.res_df.loc[ret[0]]

In [28]:
# # Now, look at the merged dataset
# tmp = tr_df[tr_df['idx']==ret[0].item()]
# display(tmp.iloc[:, :10])

# ge = extract_subset_fea(df=tmp, fea_list=ccl_fea_list, fea_sep='_')
# dd = extract_subset_fea(df=tmp, fea_list=drg_fea_list, fea_sep='_')

# print('ge.shape', ge.shape)
# print('dd.shape', dd.shape)

# Define NN
Look at "Constructing and initializing neural networks" in github.com/xduan7/UnoPytorch/blob/master/uno_pytorch.py

In [15]:
def get_model_device(model):
    return str(next(model.parameters()).device)

In [16]:
def weight_init_linear(m: nn.Module):
    # Weight initialization
    """
    Pytorch initializes the layers by default (e.g., Linear uses kaiming_uniform_)
    www.deeplearningwizard.com/deep_learning/boosting_models_pytorch/weight_initialization_activation_functions/
    stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch
    github.com/xduan7/UnoPytorch/blob/master/networks/initialization/weight_init.py
    """
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [17]:
def train():
    pass

In [18]:
def val():
    pass

### 1. Define NN_REG (Top6)

In [19]:
class NN_Reg_Merged(nn.Module):
    """ ... """
    def __init__(self,
                 ccl_dim: int,
                 drg_dim: int,
                 dr_rate: float=0.2,
                 # hidden_activation: str=None,
                 # output_activation: str=None
                ):
        super().__init__()
        self.ccl_dim = ccl_dim
        self.drg_dim = drg_dim
        self.dr_rate = dr_rate        
        
        self.fc1 = nn.Linear(self.ccl_dim + self.drg_dim, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 250)
        self.fc4 = nn.Linear(250, 60)
        self.fc5 = nn.Linear(60, 1)
        self.dropout = nn.Dropout(dr_rate)
        
        
    def forward(self, ccl_fea, drg_fea):
        x = torch.cat((ccl_fea, drg_fea), dim=1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))
        x = F.relu(self.fc5(x))
        return x

### 2. Define DrugRespNet

In [34]:
class NN_Reg_Raw(nn.Module):
    def __init__(self,
                 ccl_dim: int,
                 drg_dim: int,
                 dr_rate: float=0.2,
                 hidden_activation: str=None,
                 output_activation: str=None):
        super().__init__()
        self.ccl_dim = ccl_dim
        self.drg_dim = drg_dim
        self.dr_rate = dr_rate
        # self.hidden_activation = hidden_activation
        # self.output_activation = output_activation

        # self.layers = [self.ccl_dim+self.drg_dim, 1000, 1000, 500, 250, 125, 60, 30, 1]
        # self.net = nn.ModuleList([ nn.Linear(self.layers[i], self.layers[i+1]) for i in range(len(self.layers)-1) ])
        # self.net.apply(weight_init_linear)

        self.fc1 = nn.Linear(self.ccl_dim + self.drg_dim, 1000)
        self.bn1 = nn.BatchNorm1d(1000)
        
        self.fc2 = nn.Linear(1000, 1000)
        self.bn2 = nn.BatchNorm1d(1000)
        
        self.fc3 = nn.Linear(1000, 500)
        self.bn3 = nn.BatchNorm1d(500)
        
        self.fc4 = nn.Linear(500, 250)
        self.bn4 = nn.BatchNorm1d(250)
        
        self.fc5 = nn.Linear(250, 125)
        self.bn5 = nn.BatchNorm1d(125)
        
        self.fc6 = nn.Linear(125, 60)
        self.bn6 = nn.BatchNorm1d(60)

        self.fc7 = nn.Linear(60, 30)
        self.bn7 = nn.BatchNorm1d(30)
        
        self.fc8 = nn.Linear(30, 1)

         
    def forward(self, ccl_fea, drg_fea):
        # pytorch.org/docs/stable/nn.html?highlight=batch_norm#torch.nn.BatchNorm1d
        # pytorch.org/docs/stable/nn.html?highlight=batch_norm
        # Input
        x = torch.cat((ccl_fea, drg_fea), dim=1)
        
        # Expanded
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dr_rate, training=self.training)

        x = self.fc3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dr_rate, training=self.training)

        x = self.fc4(x)
        x = self.bn4(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dr_rate, training=self.training)
        
        x = self.fc5(x)
        x = self.bn5(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dr_rate, training=self.training)

        x = self.fc6(x)
        x = self.bn6(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dr_rate, training=self.training)

        x = self.fc7(x)
        x = self.bn7(x)
        x = F.relu(x)
        x = F.dropout(x, p=self.dr_rate, training=self.training)

        x = self.fc8(x)
        x = F.relu(x)
        
#         # Packed
#         x = F.relu(self.bn1(self.fc1(x)))
#         x = F.dropout(F.relu(self.bn2(self.fc2(x))), p=self.dr_rate, training=self.training)
#         x = F.dropout(F.relu(self.bn3(self.fc3(x))), p=self.dr_rate, training=self.training)
#         x = F.dropout(F.relu(self.bn4(self.fc4(x))), p=self.dr_rate, training=self.training)
#         x = F.dropout(F.relu(self.bn5(self.fc5(x))), p=self.dr_rate, training=self.training)
#         x = F.dropout(F.relu(self.bn6(self.fc6(x))), p=self.dr_rate, training=self.training)
#         x = F.dropout(F.relu(self.bn7(self.fc7(x))), p=self.dr_rate, training=self.training)
#         x = F.relu(self.fc8(x))
            
#         # Coded
#         for i, layer in enumerate(self.net):
#             if i == 0:
#                 # input layer
#                 x = layer(x)
#                 x = F.batch_norm(x, training=self.training)
#                 x = F.relu(x)
                
#             elif i == (len(self.layers) - 1):
#                 # output layer
#                 x = layer(x)
#                 x = F.relu(x) # TODO: should this be mse

#             else:
#                 # hidden layers
#                 x = layer(x)
#                 x = F.batch_norm(x, training=self.training)
#                 x = F.relu(x)
#                 x = F.dropout(x, p=self.dr_rate, training=self.training)
                        
        return x

### CUDA

In [20]:
# pytorch.org/docs/stable/cuda.html
# towardsdatascience.com/speed-up-your-algorithms-part-1-pytorch-56d8a4ae7051
print('is_available:  ', torch.cuda.is_available())
print('device_name:   ', torch.cuda.get_device_name(0))
print('device_count:  ', torch.cuda.device_count())
print('current_device:', torch.cuda.current_device())

is_available:   True
device_name:    GeForce RTX 2080 Ti
device_count:   4
current_device: 0


In [21]:
# Define which device to use
device_name = 'cuda:3'
device = torch.device(device_name)

### Define network and move it to device

In [22]:
model_name = 'nn_reg_merged' # drug_resp_net

if model_name == 'nn_reg_merged':
    model = NN_Reg_Merged(
        ccl_dim = tr_ds.ccl_dim,
        drg_dim = tr_ds.drg_dim,
        dr_rate = 0.2)

elif model_name == 'nn_reg_raw':
    model = NN_Reg_Raw(
        ccl_dim = tr_ds.ccl_dim,
        drg_dim = tr_ds.drg_dim,
        dr_rate = 0.2)

# Init weights
model.apply(weight_init_linear)

# Model to gpu/cpu device
model.to(device)

print(model)

NN_REG(
  (fc1): Linear(in_features=4271, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=250, bias=True)
  (fc4): Linear(in_features=250, out_features=60, bias=True)
  (fc5): Linear(in_features=60, out_features=1, bias=True)
  (dropout): Dropout(p=0.2)
)


In [23]:
# Query device where the model is
print(get_model_device(model))
print('current_device:', torch.cuda.current_device()) # why current device is 0??

cuda:3
current_device: 0


### --- Test --- forward pass of a single batch from dataloader

In [39]:
# idx = ret[0]
# src = ret[1]
# cell_id = ret[2]
# drug_id = ret[3]
# auc = ret[4]
# ccl_fea = ret[5]
# drg_fea = ret[6]

# display(ret)

In [40]:
# x = torch.cat((ccl_fea, drg_fea), dim=1)

# # Expanded
# x = model.fc1(x)
# x = model.bn1(x)
# x = F.relu(x)

# x = model.fc2(x)
# x = model.bn2(x)
# x = F.relu(x)
# x = F.dropout(x, p=model.dr_rate, training=model.training)

# x = model.fc3(x)
# x = model.bn3(x)
# x = F.relu(x)
# x = F.dropout(x, p=model.dr_rate, training=model.training)

# x = model.fc4(x)
# x = model.bn4(x)
# x = F.relu(x)
# x = F.dropout(x, p=model.dr_rate, training=model.training)

# x = model.fc5(x)
# x = model.bn5(x)
# x = F.relu(x)
# x = F.dropout(x, p=model.dr_rate, training=model.training)

# x = model.fc6(x)
# x = model.bn6(x)
# x = F.relu(x)
# x = F.dropout(x, p=model.dr_rate, training=model.training)

# x = model.fc7(x)
# x = model.bn7(x)
# x = F.relu(x)
# x = F.dropout(x, p=model.dr_rate, training=model.training)

# x = model.fc8(x)
# x = F.relu(x)

# print(x)

# Define optimer, learning rate decay
Optimizers, learning rate decay, and miscellaneous in github.com/xduan7/UnoPytorch/blob/master/uno_pytorch.py

In [24]:
# Define optimizer
# resp_opt = get_opt(opt_type=args.resp_opt, networks=resp_net, learning_rate=args.resp_lr, l2_regularization=args.l2_regularization)
opt = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)  # pytorch.org/docs/stable/optim.html

In [25]:
# Learning rate scheduler
# resp_lr_decay = LambdaLR(optimizer=resp_opt, lr_lambda=lambda e: args.lr_decay_factor ** e)

# CLR (pytorch.org/docs/stable/optim.html)
# lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=opt, base_lr=1e-4, max_lr=1e-3, mode='triangular')

In [26]:
# resp_loss_func = F.l1_loss if args.resp_loss_func == 'l1' else F.mse_loss
loss_fnc = nn.MSELoss(reduction='mean')

### --- Test --- try a single training iteration

In [27]:
idx, src, cell_id, drug_id, auc, ccl_fea, drg_fea = next(iter(tr_loader))  # (idx, SOURCE, ccl_id, drg_id, auc, ccl_fea, drg_fea)

auc = auc.to(device)
ccl_fea = ccl_fea.to(device)
drg_fea = drg_fea.to(device)
fea_dct = {'ccl_fea': ccl_fea, 'drg_fea': drg_fea}

print('ccl_fea.dtype:', ccl_fea.dtype)
print('drg_fea.dtype:', drg_fea.dtype)
print('auc.dtype:', auc.dtype)
print('')

# Forward
opt.zero_grad()
# pred = model(ccl_fea=ccl_fea, drg_fea=drg_fea)
pred = model(**fea_dct)

print(f'pred.shape {pred.shape}')
print(f'auc.shape {auc.shape}')
auc = auc.view(pred.shape)
print(f'auc.shape {auc.shape}\n')

print('pred:\n', pred[:3])
print('auc:\n', auc[:3])
pred = pred.type(auc.dtype)
print('pred:\n', pred[:3])

# Backprop
loss = loss_fnc(pred, auc)
loss.backward() # compute loss gradients wrt to model parameters and inputs
opt.step()      # update model parameters;  pytorch.org/docs/stable/optim.html

ccl_fea.dtype: torch.float32
drg_fea.dtype: torch.float32
auc.dtype: torch.float64

pred.shape torch.Size([32, 1])
auc.shape torch.Size([32])
auc.shape torch.Size([32, 1])

pred:
 tensor([[0.],
        [0.],
        [0.]], device='cuda:3', grad_fn=<SliceBackward>)
auc:
 tensor([[0.7201],
        [0.9537],
        [0.8879]], device='cuda:3', dtype=torch.float64)
pred:
 tensor([[0.],
        [0.],
        [0.]], device='cuda:3', dtype=torch.float64, grad_fn=<SliceBackward>)


# Training loops
Training/validation loops in github.com/xduan7/UnoPytorch/blob/master/uno_pytorch.py

In [56]:
# val_cl_clf_acc = []
# val_drug_target_acc = []
# val_drug_qed_mse, val_drug_qed_mae, val_drug_qed_r2 = [], [], []
val_resp_mse, val_resp_mae, val_resp_r2 = [], [], []

best_r2 = -np.inf
patience = 0
start_time = time()

In [57]:
# Create folder for validation results if not exist
# os.makedirs(args.val_results_dir)

In [ ]:
# Early stopping is decided on the validation set with the same data source as the training dataloader
# val_index = 0
# for idx, loader in enumerate(drug_resp_val_loaders):
#     if loader.dataset.data_source == args.trn_src:
#         val_index = idx

In [ ]:
max_epochs = 2
for epoch in range(max_epochs): # args.max_num_epochs

    print('=' * 80 + '\nTraining Epoch %3i:' % (epoch + 1))
    epoch_start_time = time()
    # resp_lr_decay.step(epoch)
    lr_scheduler.step(epoch)
    
    train_resp(
        device = device,
        resp_net = resp_net,
        data_loader = drug_resp_trn_loader,
        max_num_batches = args.max_num_batches,
        loss_func = resp_loss_func,
        optimizer = resp_opt)

    print('\nValidation Results:')
    
    if epoch >= args.resp_val_start_epoch:
        
        resp_mse, resp_mae, resp_r2 = valid_resp(
            epoch=epoch,
            trn_src=args.trn_src,
            device=device,
            resp_net=resp_net,
            data_loaders=drug_resp_val_loaders,
            resp_uq=args.resp_uq,
            resp_uq_dropout=args.resp_uq_dropout,
            resp_uq_num_runs=args.resp_uq_num_runs,
            val_results_dir=args.val_results_dir)

        # Save the validation results in nested list
        val_resp_mse.append(resp_mse)
        val_resp_mae.append(resp_mae)
        val_resp_r2.append(resp_r2)

        # Record the best R2 score (same data source)
        # and check for early stopping if no improvement for epochs
        if resp_r2[val_index] > best_r2:
            patience = 0
            best_r2 = resp_r2[val_index]
        else:
            patience += 1
        if patience >= args.early_stop_patience:
            print('Validation results does not improve for %d epochs ... '
                  'invoking early stopping.' % patience)
            break

    print('Epoch Running Time: %.1f Seconds.' % (time.time() - epoch_start_time))  

In [ ]:
val_resp_r2 = np.array(val_resp_r2)
val_resp_mse, val_resp_mae, val_resp_r2 = \
    np.array(val_resp_mse).reshape(-1, len(args.val_srcs)), \
    np.array(val_resp_mae).reshape(-1, len(args.val_srcs)), \
    np.array(val_resp_r2).reshape(-1, len(args.val_srcs))

print('Program Running Time: %.1f Seconds.' % (time.time() - start_time))  

# Training loops (ap)

In [28]:
def r2_torch(y_true, y_pred):
    epsilon = 1e-7  # this epsilon value used in TF
    SS_res = torch.sum( (y_true - y_pred)**2 )
    SS_tot = torch.sum( (y_true - torch.mean(y_true))**2 )
    r2 = 1 - SS_res / (SS_tot + epsilon)
    return r2

In [29]:
def update_scores_reg(pred, true, scores):
    """ Updates score metrics for regression ML predictions.
    The scores are summed for every call of the function (single func call corresponds a single batch).
    
    Note: these must be implemented with pytroch commands! Otherwise, results gives error:
    RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.   
    pred, true = pred.numpy(), yy.numpy()
    tr_mae += sklearn.metrics.mean_absolute_error(true, pred)
    tr_r2 += sklearn.metrics.r2_score(true, pred)
    """
    for m in scores.keys():
        if 'loss' in m:
            continue
            
        elif any([True if v in m else False for v in ['mean_abs_err', 'mean_absolute_error']]):
            scores[m] += torch.mean( torch.abs(pred-true) ).item()

        elif any([True if v in m else False for v in ['median_abs_err', 'median_absolute_error']]):
            scores[m] += torch.median( torch.abs(pred-true) ).item()

        elif any([True if v in m else False for v in ['mean_sqrd_err', 'mean_squared_error']]):
            scores[m] += torch.mean( torch.pow(pred-true, 0.5) ).item()  # or torch.mean(torch.sqrt(pred-true))
            
        elif any([True if v in m else False for v in ['r2', 'r2_score']]):
            scores[m] += r2_torch(y_true=true, y_pred=pred).item()
            
    return scores

In [33]:
def proc_batch(xx_dct, yy, model, loss_fnc, opt=None):
    """ 
    Args:
        opt (torch.optim) : no backprop is performed if optimizer is not provided (for val or test) 
    """
    pred = model(**xx_dct)
    pred = pred.type(yy.dtype)
    loss = loss_fnc(pred, yy)
    
    # Backward pass
    if opt is not None:
        opt.zero_grad()
        loss.backward()
        opt.step()
        
    return loss, pred

In [34]:
def fit(model: nn.Module,
        loss_fnc,
        opt: torch.optim,
        tr_dl: torch.utils.data.DataLoader,
        vl_dl: torch.utils.data.DataLoader=None,
        epochs: int=1,
        device: torch.device='cuda:0',
        verbose: bool=True,
        metrics=[]) -> dict:
    """ github.com/stared/livelossplot/blob/master/examples/pytorch.ipynb
    Args:
        metrics (list) : list of metric scores to log
            (available metrics: 'mean_abs_err','median_abs_err', 'mean_sqrd_err', 'r2)
    """ 
    print(f'Arg `device`: {device}')
    model.to(device)
    print('current_device:', torch.cuda.current_device())
    
    # Choose cuda device with context manager --> try using context manager!!!
    # with torch.cuda.device(device):
        
    # Create dicts to log scores
    if vl_dl is None:
        logs = OrderedDict({'loss': []})
        logs.update(OrderedDict({m: [] for m in metrics}))        
    else:
        logs = OrderedDict({'loss': [], 'val_loss': []})
        for m in metrics: logs.update(OrderedDict({m: [], 'val_'+m: []}))
    
    # Iter over epochs
    phases = ['train', 'val'] if vl_dl is not None else ['train']
    for ep in range(epochs):
        ep_t0 = time()
        # lr_scheduler.step()
        
        for ph in phases:
            if ph == 'train':
                model.train()
                dl = tr_dl
                scores = {m: 0 for m in logs.keys() if 'val' not in m}
                loss_name = 'loss'
            elif ph == 'val':
                model.eval()
                dl = vl_dl
                scores = {m: 0 for m in logs.keys() if 'val' in m}
                loss_name = 'val_loss'

            # Iter over batches
            for _, _, _, _, auc, ccl_fea, drg_fea in dl:
                # discuss.pytorch.org/t/expected-object-of-scalar-type-long-but-got-scalar-type-float-for-argument-2-target/33102
                auc = auc.to(device, dtype=torch.float32) # dtype=torch.float32 fixed an error
                ccl_fea = ccl_fea.to(device)
                drg_fea = drg_fea.to(device)
                fea_dct = {'ccl_fea': ccl_fea, 'drg_fea': drg_fea}
                
                # Process batch
#                 if ph == 'train':
#                     loss, pred = proc_batch(xx_dct=fea_dct, yy=auc, model=model, loss_fnc=loss_fnc, opt=opt)
#                 else:
#                     loss, pred = proc_batch(xx_dct=fea_dct, yy=auc, model=model, loss_fnc=loss_fnc, opt=None)                

                pred = model(ccl_fea=ccl_fea, drg_fea=drg_fea).type(auc.dtype)
                loss = loss_fnc(pred, auc)
                if opt is not None:
                    opt.zero_grad()
                    loss.backward()
                    opt.step()
                
                # Compute metrics (running avg)
                scores[loss_name] += loss.item()
                scores = update_scores_reg(pred=pred, true=auc, scores=scores)
            
            # Log scores
            for m in scores.keys():
                logs[m].append(scores[m]/len(dl))
            
        # del xx, yy, loss, pred, scores
        del ccl_fea, drg_fea, auc, loss, pred, scores
        
        if verbose:
            print(f'Epoch {ep+1}/{epochs}; ',
                  f'{int(time()-ep_t0)}s; ',
                  [f'{k}: {v[-1]:.3f}' for k, v in logs.items()])
            
        # TODO: log scores into file

    return logs

In [35]:
metrics = ['mean_abs_err', 'r2']
device = 'cuda:3'
verbose = True
epochs = 30
fit_kwargs = {'epochs': epochs, 'device': device, 'verbose': verbose, 'metrics': metrics}

logs = fit(model=model,
           loss_fnc=loss_fnc,
           opt=opt,
           tr_dl=tr_loader,
           vl_dl=vl_loader,
           **fit_kwargs)

Arg `device`: cuda:3
current_device: 0
Epoch 1/30;  15s;  ['loss: 0.113', 'val_loss: 0.036', 'mean_abs_err: 0.253', 'val_mean_abs_err: 0.155', 'r2: -242.482', 'val_r2: -193.656']
Epoch 2/30;  14s;  ['loss: 0.042', 'val_loss: 0.030', 'mean_abs_err: 0.162', 'val_mean_abs_err: 0.140', 'r2: -983.738', 'val_r2: -159.400']
Epoch 3/30;  15s;  ['loss: 0.034', 'val_loss: 0.028', 'mean_abs_err: 0.145', 'val_mean_abs_err: 0.133', 'r2: -48.536', 'val_r2: -146.633']
Epoch 4/30;  14s;  ['loss: 0.031', 'val_loss: 0.028', 'mean_abs_err: 0.137', 'val_mean_abs_err: 0.131', 'r2: -61.513', 'val_r2: -143.833']
Epoch 5/30;  14s;  ['loss: 0.029', 'val_loss: 0.027', 'mean_abs_err: 0.133', 'val_mean_abs_err: 0.130', 'r2: -156.263', 'val_r2: -141.217']
Epoch 6/30;  15s;  ['loss: 0.028', 'val_loss: 0.027', 'mean_abs_err: 0.131', 'val_mean_abs_err: 0.128', 'r2: -83.337', 'val_r2: -139.741']
Epoch 7/30;  14s;  ['loss: 0.028', 'val_loss: 0.027', 'mean_abs_err: 0.130', 'val_mean_abs_err: 0.128', 'r2: -39.561', 'val_

In [ ]:
plt.figure(figsize=(8,6))

# plt.plot(logs['loss'], label='loss');
# plt.plot(logs['val_loss'], label='val_loss');
# plt.legend(loc='best');

# plt.plot(logs['mean_abs_err'], label='mean_abs_err');
# plt.plot(logs['val_mean_abs_err'], label='val_mean_abs_err');
# plt.legend(loc='best');

plt.plot(logs['r2'], label='r2');
plt.plot(logs['val_r2'], label='val_r2');
plt.legend(loc='best');

plt.legend(loc='best')
plt.grid(True)